<a href="https://colab.research.google.com/github/cemalkoraybingol/2024-TUD-Capston-RAG-for-Energy-Renovations/blob/main/Energy_Dataset_to_Embedding_Retrieve_colab_w1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Capstone Applied AI Project 2024-2025**

## Leveraging Large Language Models for Policy Document Retrieval information in Energy Renovations

C. Koray Bingol, Architecture (Design and Construction Management), C.K.Bingol@tudelft.nl, cbingol@tudelft.nl

Amir Homayounirad, EEMCS (Interactive Intelligence Group), A.Homayounirad@tudelft.nl, ahomayounirad@tudelft.nl


Luciano Cavalcante Siebert, EEMCS (Interactive Intelligence Group), L.CavalcanteSiebert@tudelft.nl

Tong Wang, Architecture (Design and Construction Management), T.Wang-12@tudelft.nl

In [1]:
!pip install datasets
!pip install sentence_transformers
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=02558ad7689f76b0c7bc1a15b

In [2]:
import os
import requests
import random

#pdf processing , this is required if the text source is PDFs
#import pymupdf

#visualization of the progress
import tqdm

#huggingface to dataframes
import datasets
import pandas as pd


# **Get the Dataset**
## Load the Dataset from the disk or Download it from HuggingFace

It is easier to parse the .parquet file than download it, but in a colab environment, it is easier to download it from HuggingFace



## If needed, we can use the HuggingFace library to get the data

Here we will download the dataset from the HuggingFace

It is a memory intesive task to parse it into a pd dataframe

After the download, we will parse it into a dictionary woth Panda

Later not to repeat this process, we will save it to CSV file.

## There are two datasets

"vidore/syntheticDocQA_energy_test_ocr_chunk" > This one has the Query and Answer Pairs

"vidore/syntheticDocQA_energy_test" > This one has text description and query pairs



In [3]:
from datasets import load_dataset

# Load the datasets
dataset_1 = load_dataset("vidore/syntheticDocQA_energy_test_ocr_chunk")
dataset_2 = load_dataset("vidore/syntheticDocQA_energy_test")


dataset_1 = dataset_1.remove_columns("image")
dataset_2 = dataset_2.remove_columns("image")

print(dataset_1.keys())

print(dataset_2.keys())

for split_name in dataset_1.keys():
    print(f"Size of {split_name} set: {len(dataset_1[split_name])} rows")

for split_name in dataset_2.keys():
    print(f"Size of {split_name} set: {len(dataset_2[split_name])} rows")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/692 [00:00<?, ?B/s]

test-00000-of-00002.parquet:   0%|          | 0.00/168M [00:00<?, ?B/s]

test-00001-of-00002.parquet:   0%|          | 0.00/179M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2289 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/3.46k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/274M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

dict_keys(['test'])
dict_keys(['test'])
Size of test set: 2289 rows
Size of test set: 1000 rows


In [4]:
# Dataframe OCR
df_ocr  = pd.DataFrame(dataset_1['test'][:]) #convert the dataset 1 into a Pandas dataframe

df_ocr = df_ocr.loc[df_ocr['query'].notnull()] #there are rows without the query, to clean them.

print(len(df_ocr))
#print(df_ocr.head())

229


In [ ]:
# Dataframe QA
df_qa  = pd.DataFrame(dataset_2['test'][:]) #convert the dataset 2 into a Pandas dataframe

df_qa = df_qa.loc[df_qa['query'].notnull()] #there are rows without the query, to clean them.

# Preview the datasets
print(len(df_qa))
print(df_qa.head())
print(df_qa.loc[2, 'query'])



100
                                               query  \
0  What types of road improvements are planned fo...   
1  How did the average miles per shipment for sin...   
2  What is the total greenhouse gas reduction in ...   
3  What is the importance of accurate estimation ...   
4  What are some potential vulnerabilities of und...   

                                      image_filename  \
0  data/scrapped_pdfs_split/pages_extracted/energ...   
1  data/scrapped_pdfs_split/pages_extracted/energ...   
2  data/scrapped_pdfs_split/pages_extracted/energ...   
3  data/scrapped_pdfs_split/pages_extracted/energ...   
4  data/scrapped_pdfs_split/pages_extracted/energ...   

                                              answer page   model  \
0  ['Additional lanes', 'Standard overlay/ADA imp...  841  sonnet   
1  ['For truck, it increased from 144 to 206 mile...  153  sonnet   
2                                         ['80,829']   37  sonnet   
3  ['All around the world, system operators an

In [6]:
# Check if df_ocr is not empty
if not df_ocr.empty:
    headers = df_ocr.columns  # Get column names directly from DataFrame
    print("Headers in df_ocr:", headers)

# Check if df_qa is not empty
if not df_qa.empty:
    headers = df_qa.columns  # Get column names directly from DataFrame
    print("Headers in df_qa:", headers)


Headers in df_ocr: Index(['query', 'image_filename', 'answer', 'page', 'model', 'prompt',
       'source', 'text_description', 'chunk_id', 'chunk_type', 'chunk_image'],
      dtype='object')
Headers in df_qa: Index(['query', 'image_filename', 'answer', 'page', 'model', 'prompt',
       'source'],
      dtype='object')


In [ ]:
# Checking what kind of chunks are in the OCR dataset

match_table= 0
match_text= 0
match_figure= 0
for i in range(len(df_qa)):
    #print(df_qa.loc[i, 'query'])
    for j in range(len(df_ocr)):
        if  df_ocr.loc[j, 'query'] == 'None':
            continue
        if df_qa.loc[i, 'query'] == df_ocr.loc[j, 'query']:
            if df_ocr.loc[j, 'chunk_type'] == 'table':
                #print('table', df_ocr.loc[j, 'query'], i, j)
                match_table += 1
            elif df_ocr.loc[j, 'chunk_type'] == 'text':
                #print('text', df_ocr.loc[j, 'query'], i, j)
                match_text += 1
            elif df_ocr.loc[j, 'chunk_type'] == 'figure':
                #print('figure',  df_ocr.loc[j, 'query'], i, j)
                match_figure += 1
            elif df_ocr.loc[j, 'chunk_type'] == 'None':
                print('None')



print(match_table, match_text, match_figure)


40 135 54


In [ ]:
# Checking what are the types
print(type(df_ocr['text_description']))

text_description_ocr = []  # Define the list before using it
for item in df_ocr['text_description']:
    text_description_ocr.append(item)

print(type(text_description_ocr))


df = pd.DataFrame(text_description_ocr)
df.describe().round() # Chekcing the sizes

<class 'pandas.core.series.Series'>
<class 'list'>


,0
count,229
unique,168
top,
freq,13


In [ ]:
# Write DataFrame to CSV file without the index column (NOT MANDATORY), but sometimes useful to load from drive; it's faster.
#df_ocr.to_csv('df_ocr.csv', index=False)
#df_qa.to_csv('df_qa.csv', index=False)

df_ocr.describe().round(2) # Chekcing the samples

# Extract chunk_id and text (e.g., 'query' or 'answer', depending on the text you want to embed)
text_data = df_ocr[['query', 'text_description']].dropna()  # Replace 'query' with your relevant text column if needed
print(len(text_data))

text_data.head()


229


,query,text_description
0,What types of road improvements are planned fo...,FY 2024-2029 Rebuild Illinois Highway Improvem...
1,What types of road improvements are planned fo...,Route / Street Location Improvements Est. Cost...
2,How did the average miles per shipment for sin...,5-23\n\nIndustries covered by the 2007 Commodi...
3,How did the average miles per shipment for sin...,Ton-miles Average miles per shipment Average a...
4,How did the average miles per shipment for sin...,"Source:\n\nU.S. Department of Transportation, ..."


In [ ]:
query_data = df_qa[[ 'query', 'answer']].dropna()  # Drop the row if 'query' or 'answer' is missing
print(len(query_data))

query_data.head()


100


,query,answer
0,What types of road improvements are planned fo...,"['Additional lanes', 'Standard overlay/ADA imp..."
1,How did the average miles per shipment for sin...,"['For truck, it increased from 144 to 206 mile..."
2,What is the total greenhouse gas reduction in ...,"['80,829']"
3,What is the importance of accurate estimation ...,"['All around the world, system operators and u..."
4,What are some potential vulnerabilities of und...,"['vulnerability to extreme seismic events', 'p..."


In [ ]:
from sentence_transformers import SentenceTransformer

# Load the sentence transformer model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2') # can be any model from Hugging Face's model hub


# Create a list of texts to embed
texts = text_data['text_description'].tolist()  # Replace 'query' if using a different text column
print(len(texts), type(texts))
queries = query_data['query'].tolist()  # Replace 'query' if using a different text column
print(len(queries), type(queries[0]), print(queries[0])) # Checking the first query, length and type of the list    



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

229 <class 'list'>
What types of road improvements are planned for Greenmount Road in Belleville, Illinois?
100 <class 'str'> None


In [ ]:

# Embed the texts using the model, tokenizing the texts and converting them to PyTorch tensors
embeddings_text = model.encode(texts, convert_to_tensor=True)
print(len(embeddings_text)) # for text description
embeddings_query = model.encode(queries, convert_to_tensor=True)
print(len(embeddings_query)) # for query
#this operation will take a while (10 minutes on a Dell laptop, Processor 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz   1.80 GHz , RAM	16.0 GB (15.7 GB usable))

229
100


In [ ]:

# Combine the chunk_id with their embeddings
text_ids = text_data['query'].tolist()
embeddings_text_zip = list(zip(text_ids, embeddings_text))

# Combine the query_id with their embeddings
query_ids = query_data['query'].tolist()
answers = query_data['answer'].tolist()
embeddings_query_zip = list(zip(query_ids, embeddings_query, answers))
print(embeddings_query_zip[3])

('What is the importance of accurate estimation of available PV power according to the text?', tensor([-4.2721e-03,  6.2407e-03, -2.8245e-02, -2.5213e-03, -9.2514e-03,
         1.0749e-03,  1.2152e-02, -2.9687e-02,  9.7002e-03,  5.5950e-02,
         4.0954e-02,  1.1861e-02,  8.8275e-02, -1.7059e-02,  5.7163e-02,
        -3.8508e-02,  7.0795e-02,  2.6310e-02,  2.6419e-02,  2.5343e-02,
        -5.4225e-02, -6.6921e-02, -9.8687e-03,  5.9568e-02, -1.7400e-02,
        -2.0868e-02, -6.3650e-03,  1.4713e-04,  2.5945e-04, -7.2396e-02,
        -5.7286e-03, -6.0334e-02, -8.4898e-03, -1.1184e-02,  9.2341e-07,
        -2.1534e-02, -1.7078e-02,  2.1828e-02,  1.7340e-02,  7.1878e-02,
         4.1692e-02, -9.5746e-03,  1.9111e-02, -2.1683e-02,  5.7723e-02,
        -1.4764e-02,  5.8327e-02,  1.9711e-03, -5.4801e-02, -4.0309e-03,
         8.6761e-03,  1.1808e-01, -2.0346e-02, -1.6296e-02, -1.8749e-03,
        -3.8855e-02, -5.0068e-03, -1.3330e-02,  9.3574e-03,  2.9413e-02,
        -3.3628e-02,  7.4959e

## Packing the embeddings as a .pkl file to load later from the drive. 

This step helps you to save computing time after you finish your embedding. If you save the embeddings as a file then, or update them regularly as you progress then the team will just download these file to work on their part.

In [16]:
import pickle

# Save the chunk embeddings to a file
with open('text_embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings_text_zip, f)

    # Save the chunk embeddings to a file
with open('query_embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings_query_zip, f)

## Load the Embeddings from a File

This is a workflow you need to solve, it doesn't have to me scalable at the moment. So, you may use conventional file sharing systems to deal with it.

In [17]:


import pickle

# Load text embeddings
with open('text_embeddings.pkl', 'rb') as f:
    text_embeddings = pickle.load(f)

# Load query embeddings (if saved separately)
with open('query_embeddings.pkl', 'rb') as f:
    query_embeddings = pickle.load(f)

# ---------------------------------------------------------------------------
# MemoryError: Beware that this load is not optimized for large files, we need other methods to load the embeddings
# ---------------------------------------------------------------------------

## Recap

Here we have two sets; one has description texts and queries related to these texts. The other one is the query and answer pairs. The main thing to navigate between them is the 'query'; it is the unique identifier.

The following process is to run similarity check between the description texts and queries in different sets. You'll find the most suitable method and implement it anyways.

The results will be a list of query, description and answer.

In [18]:
import numpy as np

# Extract chunk IDs and their embeddings
#text_ids are the 'queries'
#text_vectors are the embeddings
text_ids, text_vectors = zip(*embeddings_text_zip)
print(len(text_vectors), len(text_ids))
print(len(text_vectors[0]), len(text_ids[0]))
text_vectors = np.array([embeddings_text.cpu().numpy() for embeddings_text in text_vectors])  # Convert to NumPy array

# Extract query IDs and their embeddings
#query_ids are the 'queries'
#query_vectors are the embeddings of the queries
query_ids, query_vectors, answers = zip(*embeddings_query_zip)
print(len(query_vectors), len(query_ids))
print(len(query_vectors[0]), len(query_ids[0]))

query_vectors = np.array([embeddings_query.cpu().numpy() for embeddings_query in query_vectors])  # Convert to NumPy array


print(query_vectors.shape)
#print(query_ids[0])
#print(query_vectors[0])
print(text_vectors.shape)
text_vectors.shape


# This might also take a while, depending on the size of the embeddings

229 229
768 88
100 100
768 88
(100, 768)
(229, 768)


(229, 768)

In [19]:
# Ensure the length of text_vectors matches the length of text_ids
print(f"Number of text vectors: {len(text_vectors)}")
print(f"Number of text IDs: {len(text_ids)}")

print(f"Length of text_vectors: {len(text_vectors)}")
print(f"Length of text_ids: {len(text_ids)}")

Number of text vectors: 229
Number of text IDs: 229
Length of text_vectors: 229
Length of text_ids: 229


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Compute the cosine similarity between query and text vectors
similarity_matrix = cosine_similarity(query_vectors, text_vectors) # The shape should be Queries+Answers x Texts+Queries , 100 x 229

# similarity_matrix[i, j] gives the cosine similarity between query i and text j
print(similarity_matrix.shape)

# Sample output
k= 2
m= 3
print(similarity_matrix[k][m])
print(queries[k])
print(texts[m])




(100, 229)
0.452396
What is the total greenhouse gas reduction in metric tons projected for 2050 across all measures listed?
Ton-miles Average miles per shipment Average annual Average percent annual 1997 2002 2007 change percent Mode of transportation (billions) (billions) (billions) — (1997-2007) 1997 2002 2007 change All modes 2,661.4 3,137.9 3,344.7 2.3% 472 546 619 2.7% Single modes 2,383.5 2,867.9 2,894.3 2.0% 184 240 234 2.4% Truck* 1,023.5 1,255.9 1,342.1 2.7% 144 173 206 3.6% For-hire truck 741.1 959.6 1,055.6 3.6% 485 523 599 2.1% Private truck 268.6 291.1 286.5 0.6% 53 64 57 0.7% Rail 1,022.5 1,261.6 1,344.0 2.8% 769 807 728 -0.5% Water 261.7 282.7 157.3 -5.0% 482 568 520 0.8% Shallow draft 189.3 211.5 117.5 -4.7% 177 450 144 -2.0% Great Lakes 13.4 13.8 6.9 -6.4% 204 339 657 12.4% Deep draft 59.0 57.4 33.0 -0.1% 1,024 664 923 -1.0% Air (includes truck and air) 6.2 5.8 4.5 -3.2% 1,380 1,919 1,304 -0.6% Pipeline? G c c c c c © c Multiple modes 204.5 225.7 416.6 7.4% 813 895 97

In [ ]:
# Sample output
k= 0
m= 0
print(len(embeddings_query_zip),embeddings_query_zip[k][0], type(embeddings_query_zip[k][1])) # the query and the embedding of the query
print(len(embeddings_text_zip), embeddings_text_zip[k][0], type(embeddings_text_zip[k][1])) # the query and the embedding of the text related to the query
print("similarity matrix: ", similarity_matrix.shape,similarity_matrix[k][m]) # similarity score between the queries and the texts, the identifier is the indicies of the queries and the texts
print(query_data.iloc[0]) #
print(text_data.iloc[0])

100 What types of road improvements are planned for Greenmount Road in Belleville, Illinois? <class 'torch.Tensor'>
229 What types of road improvements are planned for Greenmount Road in Belleville, Illinois? <class 'torch.Tensor'>
similarity matrix:  (100, 229) 0.59785545
query     What types of road improvements are planned fo...
answer    ['Additional lanes', 'Standard overlay/ADA imp...
Name: 0, dtype: object
query               What types of road improvements are planned fo...
text_description    FY 2024-2029 Rebuild Illinois Highway Improvem...
Name: 0, dtype: object


In [ ]:
top_n = 5  # Number of most relevant texts you want to find (top 5 or top 10, etc.)

# Store the top 3 most similar text IDs for each query
top_matches = {}
# Creates a list containing 5 lists, each of 8 items, all set to 0
match = [0] * 5
for i in range(5):
    match[i] = 0

top_indices_list = []


for i, query_id in enumerate(query_ids):
    #print(f"Processing query {i} / {query_id}")
    # Get the similarity scores for the current query
    similarities = similarity_matrix[i]
    #print(len(similarities))

    # Get the indices of the top N most similar text vectors
    top_indices = np.argsort(similarities)[-top_n:][::-1]  # Sort and get top N indices [-top_n:] gets the last N elements in reverse order, [::-1] reverses the order

    # This list is to check if the ground truth query is in the top N most similar texts, and at which position
    if query_data.iloc[i]['query'] == text_data.iloc[top_indices[0]]['query']:
        #print("MAAAAATCH!!!")
        match[0] += 1
    if query_data.iloc[i]['query'] == text_data.iloc[top_indices[1]]['query']:
        #print("2 MAAAAATCH!!!")
        match[1] += 1
    if query_data.iloc[i]['query'] == text_data.iloc[top_indices[2]]['query']:
        #print("3 MAAAAATCH!!!")
        match[2] += 1
    if query_data.iloc[i]['query'] == text_data.iloc[top_indices[3]]['query']:
        #print("4 MAAAAATCH!!!")
        match[3] += 1
    if query_data.iloc[i]['query'] == text_data.iloc[top_indices[4]]['query']:
        #print("5 MAAAAATCH!!!")
        match[4] += 1


    # This the list of the top indices of the texts that are most similar to the query, in the order of the similarity score
    top_indices_list.append(top_indices)

    ### This dict stores the queries as keys and the top matches as values with the identifer 'query' of the information text
    top_matches[query_id] = [(text_ids[idx], similarities[idx]) for idx in top_indices]


print(match) #number of matches based on the rankings



[68, 31, 16, 8, 6]


You need a huggingface token to doenload some of the models. It is free and generate one for the team. You can delete it after the project. 

In [ ]:
from huggingface_hub import login
# This 
login('A37CHARACTERHUGGINGFACETOKEN') # Log in to the Hugging Face Hub using the TOKEN

In [32]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
model_name = 'distilgpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [33]:

index_num = 20  # just and experimental num with un QA pair length, which is 100.

# GROUND TRUTH QUERY, TEXT DESCRIPTION AND ANSWER
print(query_data.iloc[index_num]['query'])
query = query_data.iloc[index_num]['query']
print(text_data.iloc[top_indices_list[index_num][0]]['text_description'])
text = text_data.iloc[top_indices_list[index_num][0]]['text_description']
print(query_data.iloc[index_num]['answer'])




Why are the strategies found within this plan meant to address transportation equity?
ELECTRIC VEHICLE INFRASTRUCTURE PLAN = EAGLE COUNTY, COLORADO

EQUITY

One of the many directives provided in the State of Colorado's Environmental Justice Act is to “promote an equitable transition to transportation electrification, zero-emission vehicles (ZEV), transportation systems, and land use patterns that reduce energy use and greenhouse gas emissions. ”” While this Act provides instruction for State agencies to remediate environmental inequities, it is also up to us, in our home communities, to ensure we are not overlooking, therefore repeating, cycles and patterns of inequality.

The writers of this plan understand that purchasing an EV and EVSE is presently cost-burdensome for many Eagle County community members. Technology is changing quickly, and reports suggest EVs will be as cheap to produce as internal combustion engines (ICE) by the latter half of the decade *. Even still, not everyon

In [ ]:
# A sample prompt genereation function

def create_single_prompt(query, description_text):
    # Construct the prompt using the query and description_text
    prompt = f"Question: {query}\nContext: {description_text}\nAnswer:"
    return prompt

# Example usage
query = "What is renewable energy?"
description_text = "Renewable energy comes from natural sources like the sun, wind, and water."

# Generate the prompt
prompt = create_single_prompt(query, text)

# Display the prompt
print(prompt)



Question: What is renewable energy?
Context: ELECTRIC VEHICLE INFRASTRUCTURE PLAN = EAGLE COUNTY, COLORADO

EQUITY

One of the many directives provided in the State of Colorado's Environmental Justice Act is to “promote an equitable transition to transportation electrification, zero-emission vehicles (ZEV), transportation systems, and land use patterns that reduce energy use and greenhouse gas emissions. ”” While this Act provides instruction for State agencies to remediate environmental inequities, it is also up to us, in our home communities, to ensure we are not overlooking, therefore repeating, cycles and patterns of inequality.

The writers of this plan understand that purchasing an EV and EVSE is presently cost-burdensome for many Eagle County community members. Technology is changing quickly, and reports suggest EVs will be as cheap to produce as internal combustion engines (ICE) by the latter half of the decade *. Even still, not everyone can afford a new vehicle. Additionally,

In [35]:
from transformers import set_seed

# seed for reproducibility
set_seed(42)

def generate_answer(prompt, tokenizer, model, max_new_tokens=100):
    # Encode the prompt into tokens for the model
    inputs = tokenizer.encode(prompt, return_tensors="pt")

    # Generate text based on the prompt
    outputs = model.generate(
        inputs,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    # Decode the generated text to a readable string
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Generate the answer
answer = generate_answer(prompt, tokenizer, model)
print("Generated Answer:")
print(answer)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Answer:
Question: What is renewable energy?
Context: ELECTRIC VEHICLE INFRASTRUCTURE PLAN = EAGLE COUNTY, COLORADO

EQUITY

One of the many directives provided in the State of Colorado's Environmental Justice Act is to “promote an equitable transition to transportation electrification, zero-emission vehicles (ZEV), transportation systems, and land use patterns that reduce energy use and greenhouse gas emissions. ”” While this Act provides instruction for State agencies to remediate environmental inequities, it is also up to us, in our home communities, to ensure we are not overlooking, therefore repeating, cycles and patterns of inequality.

The writers of this plan understand that purchasing an EV and EVSE is presently cost-burdensome for many Eagle County community members. Technology is changing quickly, and reports suggest EVs will be as cheap to produce as internal combustion engines (ICE) by the latter half of the decade *. Even still, not everyone can afford a new vehi

In [36]:
from rouge_score import rouge_scorer

# Example generated text and reference text
generated_text = answer
reference_text = query_data.iloc[index_num]['answer']


In [37]:
# Initialize the scorer with the types of ROUGE scores you want
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate ROUGE scores
scores = scorer.score(reference_text, generated_text)

# Display the scores
print("ROUGE Scores:")
for metric, score in scores.items():
    print(f"{metric}:")
    print(f"  Precision: {score.precision:.2f}")
    print(f"  Recall:    {score.recall:.2f}")
    print(f"  F1 Score:  {score.fmeasure:.2f}")


ROUGE Scores:
rouge1:
  Precision: 0.07
  Recall:    1.00
  F1 Score:  0.13
rouge2:
  Precision: 0.07
  Recall:    1.00
  F1 Score:  0.12
rougeL:
  Precision: 0.07
  Recall:    1.00
  F1 Score:  0.13


# Test Results
## distilgpt2
ROUGE Scores:
### rouge1:
*  Precision: 0.07
*  Recall:    1.00
*  F1 Score:  0.13
### rouge2:
*  Precision: 0.07
*  Recall:    1.00
*  F1 Score:  0.12
### rougeL:
*  Precision: 0.07
*  Recall:    1.00
*  F1 Score:  0.13